# qlanth dressed in Python

In [1]:
from sparsef1 import *
from sparsef2 import *
from sparsef3 import *
from sparsef4 import *
from sparsef5 import *
from sparsef6 import *
from sparsef7 import *
import numpy as np
from matplotlib import pyplot as plt
import inspect
import cmasher as cm
import sympy as sp
from LSJMJ_basis import LSJMJ_basis
from time import time

In [4]:
LSJMJ_basis.keys()

dict_keys(['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7'])

In [2]:
iterover = [effective_ham_f1, effective_ham_f2, effective_ham_f3, effective_ham_f4, effective_ham_f5, effective_ham_f6, effective_ham_f7]
argspecs = {}
for idx, fun in enumerate(iterover):
    argspec = inspect.getfullargspec(fun)
    argvars = ', '.join(argspec.args)
    argvars = 'f%d : ' % (idx+1) + '(' + argvars + ')'
    argspecs[idx+1] = (argvars)
print('\n'.join(list(argspecs.values())))

f1 : (B02, B04, B06, B12, B14, B16, B22, B24, B26, B34, B36, B44, B46, B56, B66, S12, S14, S16, S22, S24, S26, S34, S36, S44, S46, S56, S66, α, β, γ, ζ)
f2 : (B02, B04, B06, B12, B14, B16, B22, B24, B26, B34, B36, B44, B46, B56, B66, F0, F2, F4, F6, M0, M2, M4, P2, P4, P6, S12, S14, S16, S22, S24, S26, S34, S36, S44, S46, S56, S66, T2, t2Switch, α, β, γ, ζ, σSS)
f3 : (B02, B04, B06, B12, B14, B16, B22, B24, B26, B34, B36, B44, B46, B56, B66, F0, F2, F4, F6, M0, M2, M4, P2, P4, P6, S12, S14, S16, S22, S24, S26, S34, S36, S44, S46, S56, S66, T2, T2p, t2Switch, T3, T4, T6, T7, T8, α, β, γ, ζ, σSS)
f4 : (B02, B04, B06, B12, B14, B16, B22, B24, B26, B34, B36, B44, B46, B56, B66, F0, F2, F4, F6, M0, M2, M4, P2, P4, P6, S12, S14, S16, S22, S24, S26, S34, S36, S44, S46, S56, S66, T2, T2p, t2Switch, T3, T4, T6, T7, T8, α, β, γ, ζ, σSS)
f5 : (B02, B04, B06, B12, B14, B16, B22, B24, B26, B34, B36, B44, B46, B56, B66, F0, F2, F4, F6, M0, M2, M4, P2, P4, P6, S12, S14, S16, S22, S24, S26, S34, S36, 

In [3]:
# this loop is simply to evaluate how
# quicly the sparse matrices can be computed
iterover = [effective_ham_f1, effective_ham_f2, effective_ham_f3, effective_ham_f4, effective_ham_f5, effective_ham_f6, effective_ham_f7]
for idx, fun in enumerate(iterover):
    argspec = inspect.getfullargspec(fun)
    num_parameters = len(argspec.args)
    print("f%d" %(idx+1), "num parameters = ", num_parameters)
    params = [np.random.random() for i in range(num_parameters)]
    start_time = time()
    pymat  = fun(*params)
    pyarray = pymat.toarray()
    end_time = time()
    print("time to compute python matrix = %.2f min" % ((end_time - start_time)/60.))

f1 num parameters =  31
time to compute python matrix = 0.00 min
f2 num parameters =  44
time to compute python matrix = 0.00 min
f3 num parameters =  50
time to compute python matrix = 0.00 min
f4 num parameters =  50
time to compute python matrix = 0.03 min
f5 num parameters =  50
time to compute python matrix = 0.10 min
f6 num parameters =  50
time to compute python matrix = 0.21 min
f7 num parameters =  50
time to compute python matrix = 0.13 min


## Example : Pr in LaF3

In [3]:
fHam = effective_ham_f2(
    F0=0,    
    F2=68878,
    F4=50347,
    F6=32901,
    t2Switch=0,
    ζ=751.7,
    α=16.23,
    β=-566.6,
    γ=1371,
    M0=2.08,
    M2=0.56*2.08,
    M4=0.31*2.08,
    P2=-88.6,
    P4=-0.5*88.6,
    P6=-0.1*88.6,
    T2=0.,
    σSS=1,
    B02=-218,
    B04=738.,
    B06=679,
    B22=-120.,
    B24=431,
    B44=616,
    B26=-921,
    B46=-348,
    B66=-788
    )

In [4]:
energies = np.sort(np.linalg.eigvals(fHam.toarray()))
energies = energies - energies[0]
eigenvectors = np.linalg.eig(fHam.toarray())[1]

In [5]:
energies

array([    0.        ,    71.53017127,    94.93432674,   137.83649535,
         182.54983013,   220.98425571,   333.45927379,   444.10937084,
         462.75293024,  2132.01855019,  2163.28726492,  2196.03435496,
        2289.62760543,  2294.99111667,  2300.62904837,  2323.05003215,
        2404.2515616 ,  2417.51411698,  2442.82006161,  2545.51120848,
        4179.09718418,  4199.79750002,  4282.85561178,  4321.4547533 ,
        4384.10297602,  4467.03412033,  4478.41875303,  4496.18402198,
        4507.41039512,  4590.19909272,  4692.81913123,  4711.66894367,
        4813.7096838 ,  5144.66943793,  5182.31656837,  5184.58498389,
        5269.49141845,  5275.39111783,  6461.28648764,  6494.86675247,
        6512.32422146,  6583.92636117,  6605.34261406,  6633.20865695,
        6745.12869837,  6919.13188678,  6937.44973571,  6951.48773672,
        6953.42853137,  6955.17331907,  6984.77338034,  7035.60174026,
        7097.73404136,  7153.43581376,  9722.59767244,  9763.54001573,
      

In [87]:
# The eigenvectors are to be interpreted in the ordered
# basis provided by LSJMJ_basis
# for example the two largerst contribution to the first eigenvector is
ground_state = eigenvectors[0]
argsorter = np.argsort(np.abs(ground_state))
basis = LSJMJ_basis["f2"]
major_basis_components = [
    (np.round(ground_state[idx],3)* 
    sp.Symbol('*|%s,J=%d,MJ=%d>' % tuple(basis[idx]))) for idx in argsorter[-2:]]
sum(major_basis_components)


0.103**|3P,J=0,MJ=0> - 0.992**|3P,J=1,MJ=0>